In [1]:
from ib_insync import *
util.startLoop()
ib = IB()
ib.connect('127.0.0.1', 7497, clientId=1)
if ib.isConnected():
    print("Успешно подключились к TWS")
ib.disconnect()

Успешно подключились к TWS


In [2]:
import requests

In [3]:
ticker = str(input('Enter ticker: ')).upper()
amount = int(input('Enter amount: '))

Enter ticker: spy
Enter amount: 10


## Получение минутных данных

Загрузка данных осуществляется с помощью сервиса Alpha Vantage (https://www.alphavantage.co/). Здесь, с определенными ограничениями, можно бесплатно взять биржевые данные американских компаний. Ограничения заключаются в лимите обращений к данным, не более пяти раз в минуту и не более пятисот в сутки. Но так как робот будет делать один запрос в минуту, AV отлично для этого подойдет. Чтобы начать обращаться к API сервиса, необходимо получить API Key.

API Key: P9DHEUMVT6580QQX

In [5]:
filename = f'{ticker}.csv'
interval = '1min'
apikey = 'P9DHEUMVT6580QQX'
url = f'https://www.alphavantage.co/query?' \
      f'function=TIME_SERIES_INTRADAY&symbol={ticker}&' \
      f'interval={interval}&apikey={apikey}&datatype=csv&outputsize=compact'

r = requests.get(url)
content = r.content.decode('UTF-8')

with open(filename, "w") as file:
    file.write(content)

print(f'\tUpdate {ticker} price history:', filename)

	Update SPY price history: SPY.csv


## Описание данных

В случае необходимости, Alpha Vantage можно заменить другим источником данных. Для этого просто следует заменить url-запрос на адрес вашего сервиса. Однако формат данных должен соответствовать csv, иначе корректная работа робота не гарантируется. Например, даже если взять json данные из того же Alpha Vantage, робот не сможет прочитать их и выдаст исключение. Однако, если загрузить json данные из биржи IEX, то все будет работать и ничего переделывать не придется.
Лучшим вариантом для данного робота, будет использование данных в формате csv:

Alpha Vantage позволяет загружать
    минутные (TIME_SERIES_INTRADAY — как в нашем случае),
    дневные (TIME_SERIES_DAILY),
    недельные (TIME_SERIES_WEEKLY)
    месячные данные (TIME_SERIES_MONTHLY).

Поддерживаются следующие значения временных интервалов: 1 мин, 5 мин, 15 мин, 30 мин, 60 мин. Как наверное уже заметили ранее, данный робот использует интервал в 1 минуту.

Сервис позволяет получать данные в двух форматах: csv и json. По умолчанию выдается json, поэтому для получения csv, следует явно это указать в запросе: **datatype=csv**

Еще следует учитывать, что в Alpha Vantage есть два размера файла: compact и full. По умолчанию идет compact и означает, что файл включает в себя 100 последних значений, в нашем случае это 100 последних минут. При указании опции full, AV представит всю историю данных за сутки, если используем TIME_SERIES_INTRADAY. (**outputsize=compact**)

Например:

url = f'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={ticker}&interval={interval}&apikey={apikey}&datatype=csv&outputsize=full'
